In [ ]:
import vectorbt as vbt
import pandas as pd
import config
from strategy import *
%run strategy.py

coindata = vbt.CCXTData.download_symbol(symbol=config.COINPAIR ,exchange = config.EXCHANGE ,config = config.EXCHANGECONFIG, start="1000 days ago UTC", timeframe = "5m")
coindata = coindata[:-1]
print(coindata)
#coindata.to_csv('data/coindata.csv')

In [ ]:
tech_data = technical_strategy(coindata)
acc = manager(tech_data)

In [ ]:
entries = acc.order_test_df()['entry']
exits = acc.order_test_df()['exit']
pf_long = vbt.Portfolio.from_signals(coindata.get('Close'),entries=entries,exits=exits, init_cash=1000, fees = 0.0004)
pf_short = vbt.Portfolio.from_signals(coindata.get('Close'),short_entries=exits, short_exits=entries, init_cash=1000, fees = 0.0004)
pf_both = vbt.Portfolio.from_signals(coindata.get('Close'),entries=entries,exits=exits, short_entries=exits, short_exits=entries, init_cash=1000, fees = 0.0004)
#pf_both.plot().show()
portfolio_df = pd.concat([pf_long.stats(), pf_short.stats(), pf_both.stats()], axis=1)
portfolio_df.columns = ['Long', 'Short', 'Both']
portfolio_df.to_html('data/performance.html')
portfolio_df